In [3]:
import os
import shutil

In [4]:

def get_file_extension(file_name: str) -> str:
    """
    Extracts the file extension from the given file name.
    """
    return os.path.splitext(file_name)[1][1:] if '.' in file_name else ''

In [5]:
def create_repository_and_add_file(file_path: str) -> None:
    """
    Creates a folder named after the file extension and moves the file into it.
    If a folder with that name already exists, the file is added to that folder.

    Args:
    - file_path (str): The path to the file that needs to be moved.
    """
    # Extract file name and extension
    file_name = os.path.basename(file_path)
    file_extension = get_file_extension(file_name)

    if not file_extension:
        print("No extension found for the file.")
        return

    # Create a directory based on the file extension (if not exists)
    directory = os.path.join(os.getcwd(), file_extension)  # Creates a directory based on the extension

    if not os.path.exists(directory):
        os.makedirs(directory)  # Create directory if it does not exist
        print(f"Created new directory: {directory}")
    else:
        print(f"Directory {directory} already exists.")

    # Move the file to the created directory
    destination_path = os.path.join(directory, file_name)

    # Check if the file already exists in the directory, if so, avoid overwriting
    if os.path.exists(destination_path):
        print(f"File {file_name} already exists in the directory. Skipping...")
    else:
        shutil.move(file_path, destination_path)  # Move the file
        print(f"File {file_name} has been moved to {directory}")

In [6]:
import os
from git import Repo
from dotenv import load_dotenv
import shutil

# Load environment variables from the .env file
load_dotenv()

# Directory where repositories will be cloned (outside microservice-code-repository, directly in Plagiarism-detector)
BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..', 'repositories')
os.makedirs(BASE_DIR, exist_ok=True)

# Function to retrieve repository URLs from the environment variables
def get_repository_urls():
    """
    Retrieves the list of GitHub repository URLs from environment variables dynamically.
    """
    repo_urls = []
    
    # Loop through all environment variables to find those with REPO_URL_ prefix
    for key, value in os.environ.items():
        if key.startswith("REPO_URL_"):
            repo_urls.append(value)
    
    return repo_urls

def get_file_extension(file_name: str) -> str:
    """
    Extracts the file extension from the given file name.
    """
    return os.path.splitext(file_name)[1][1:] if '.' in file_name else ''

def create_repository_and_add_file(file_path: str) -> None:
    """
    Creates a folder named after the file extension and moves the file into it.
    If a folder with that name already exists, the file is added to that folder.
    """
    file_name = os.path.basename(file_path)
    file_extension = get_file_extension(file_name)

    if not file_extension:
        print(f"Skipping {file_name} as it has no extension.")
        return

    # Create a directory based on the file extension (if not exists)
    directory = os.path.join(os.getcwd(), file_extension)  # Creates a directory based on the extension

    if not os.path.exists(directory):
        os.makedirs(directory)  # Create directory if it does not exist
        print(f"Created new directory: {directory}")
    else:
        print(f"Directory {directory} already exists.")

    # Move the file to the created directory
    destination_path = os.path.join(directory, file_name)

    # Check if the file already exists in the directory, if so, avoid overwriting
    if os.path.exists(destination_path):
        print(f"File {file_name} already exists in the directory. Skipping...")
    else:
        shutil.move(file_path, destination_path)  # Move the file
        print(f"File {file_name} has been moved to {directory}")

def clone_repositories():
    """
    Clones repositories listed in the .env file, separates files by extension, and stores them locally.
    """
    repos = get_repository_urls()

    if not repos:
        print("No repositories found to clone.")
        return

    cloned_repos = []
    failed_repos = []

    for repo_url in repos:
        repo_name = repo_url.split("/")[-1].replace(".git", "")
        repo_path = os.path.join(BASE_DIR, repo_name)

        # Check if the repository already exists
        if os.path.exists(repo_path):
            print(f"Repository {repo_name} already cloned.")
            continue

        try:
            # Clone the repository
            Repo.clone_from(repo_url, repo_path)
            cloned_repos.append(repo_name)

            # Now go through the files and move them to the right directories
            for root, _, files in os.walk(repo_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    create_repository_and_add_file(file_path)
        except Exception as e:
            failed_repos.append(repo_name)

    if cloned_repos:
        print(f"Successfully cloned repositories: {', '.join(cloned_repos)}")
    if failed_repos:
        print(f"Failed to clone repositories: {', '.join(failed_repos)}")

def remove_repository(repo_name: str):
    """
    Removes a cloned repository.
    """
    repo_path = os.path.join(BASE_DIR, repo_name)
    if not os.path.exists(repo_path):
        print(f"Repository {repo_name} not found.")
        return

    # Remove the repository directory
    shutil.rmtree(repo_path)
    print(f"Repository {repo_name} removed successfully!")

# Run the function in the script
clone_repositories()

NameError: name '__file__' is not defined